In [23]:
!pwd

/Users/chuhsitseng/flatiron/project/sommelier/notebooks/report


In [22]:
import matplotlib.pyplot as plt
%matplotlib inline  
import pandas as pd


ModuleNotFoundError: No module named 'matplotlib'

In [13]:
# There are totla 129971 records.
wine_data = pd.read_csv("../../data/raw/winemag-data-130k-v2.csv") 

In [19]:
wine_data['country'] = wine_data['country'].fillna("Unknown")
wine_data['designation'] = wine_data['designation'].fillna("Unknown")
wine_data['province'] = wine_data['province'].fillna("Unknown")
wine_data['region_1'] = wine_data['region_1'].fillna("Unknown")
wine_data['region_2'] = wine_data['region_2'].fillna("Unknown")
wine_data['taster_name'] = wine_data['taster_name'].fillna("Unknown")
wine_data['taster_twitter_handle'] = wine_data['taster_twitter_handle'].fillna("Unknown")
wine_data['variety'] = wine_data['variety'].fillna("Unknown")
wine_data['price'] = wine_data['price'].fillna(0)

#Drop the duplicate descriptions
wine_data = wine_data.drop_duplicates('description') # 10016 duplicates

In [17]:
wine_data.shape

(119955, 14)

In [18]:
wine_data[wine_data['title'].duplicated()].shape

(1148, 14)

There are only about 1148 bottle of wine have more than one taster's review.

In [20]:
wine_data.dtypes

Unnamed: 0                 int64
country                   object
description               object
designation               object
points                     int64
price                    float64
province                  object
region_1                  object
region_2                  object
taster_name               object
taster_twitter_handle     object
title                     object
variety                   object
winery                    object
dtype: object

In [21]:
price_points = wine_data[wine_data['price'] < 100]
price_points.plot.hexbin(x='price', y='points', title= 'Review Distribution',gridsize=18,cmap=plt.cm.Blues, fontsize=10)
plt.savefig('price_points.png')

NameError: name 'plt' is not defined